In [1]:
import pandas as pd
import numpy as np
import scipy 
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings 
import os
import googlemaps
from datetime import datetime

# Setting the working directory
os.chdir(r'../')

# Pandas option:
pd.set_option('max_colwidth',40)  # This allows us to see the data upto 40 characters per each column

pd.options.display.max_columns = None   # This allows us to see all the columns without collapsing 


# The below code is for for showing all expressions, not just the result of the last expression
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Table styles, this below code is for decorating the dataframe no functional use.
table_styles = {
        'cerulean_palette' : [
        dict(selector="th", props=[("color", "#FFFFFF"), ("background", "#004D80"), ("text-transform", "capitalize")]),
        dict(selector="td", props=[("color", "#333333")]),
        dict(selector="table", props=[("font-family", 'Arial'), ("border-collapse", "collapse")]),
        dict(selector='tr:nth-child(even)', props=[('background', '#D3EEFF')]),
        dict(selector='tr:nth-child(odd)', props=[('background', '#FFFFFF')]),
        dict(selector="th", props=[("border", "1px solid #0070BA")]),
        dict(selector="td", props=[("border", "1px solid #0070BA")]),
        dict(selector="tr:hover", props=[("background", "#80D0FF")])
    ]
}

# Filtering all the warnings:
warnings.filterwarnings('ignore')

# Load Data

In [39]:
df=pd.read_csv('data/zomato.csv')
df['ID'] = [str(i).zfill(5) for i in range(1, len(df) + 1)]
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),ID
0,https://www.zomato.com/bangalore/jal...,"942, 21st Main Road, 2nd Stage, Bana...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, P...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful...",[],Buffet,Banashankari,00001
1,https://www.zomato.com/bangalore/spi...,"2nd Floor, 80 Feet Road, Near Big Ba...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirva...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been he...",[],Buffet,Banashankari,00002
2,https://www.zomato.com/SanchurroBang...,"1112, Next to KIMS Medical College, ...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is...",[],Buffet,Banashankari,00003
3,https://www.zomato.com/bangalore/add...,"1st Floor, Annakuteera, 3rd Stage, B...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food ...",[],Buffet,Banashankari,00004
4,https://www.zomato.com/bangalore/gra...,"10, 3rd Floor, Lakshmi Associates, G...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good r...",[],Buffet,Banashankari,00005


In [42]:
locations = pd.unique(df['location']).tolist()
location_ids = range(1, len(locations)+1)
location_ids = [str(i).zfill(3) for i in location_ids]
location_id_map_dict = {
    str(i): str(j) for i,j in zip(locations, location_ids)
}
df['location_id'] = df['location'].apply(lambda x: location_id_map_dict[str(x)])

In [43]:
df.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)', 'ID', 'location_id'],
      dtype='object')

In [7]:
os.getcwd()

'/Users/howechen/Project/ntu_sd6101_team_project'

# Get Geocode Data

In [2]:
# Google Maps API Methods
with open(r'../my api token/googlemaps.txt', 'r') as f:
    api_token = f.read()
    f.close()
gmaps = googlemaps.Client(key=api_token)

# Geocoding an address
# geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')

# # Look up an address with reverse geocoding
# reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))

# # Request directions via public transit
# now = datetime.now()
# directions_result = gmaps.directions("Sydney Town Hall",
#                                      "Parramatta, NSW",
#                                      mode="transit",
#                                      departure_time=now)

# # Validate an address with address validation
# addressvalidation_result =  gmaps.addressvalidation(['1600 Amphitheatre Pk'], 
#                                                     regionCode='US',
#                                                     locality='Mountain View', 
#                                                     enableUspsCass=True)

# # Get an Address Descriptor of a location in the reverse geocoding response
# # address_descriptor_result = gmaps.reverse_geocode((40.714224, -73.961452), enable_address_descriptor=True)


In [4]:
# geocode_result[0]['geometry']

In [49]:
def get_lat_lng(gmaps, address):
    try:
        geocode_result = gmaps.geocode(address)
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        return lat, lng
    except Exception as e:
        print(f'Error: {e}')
        return None, None

In [47]:
location_df = df.loc[:, ['ID', 'location_id', 'location', 'address']]
location_df.rename(columns={'location': 'location_name'}, inplace=True)
location_df.head()

,ID,location_id,location_name,address
0,00001,001,Banashankari,"942, 21st Main Road, 2nd Stage, Bana..."
1,00002,001,Banashankari,"2nd Floor, 80 Feet Road, Near Big Ba..."
2,00003,001,Banashankari,"1112, Next to KIMS Medical College, ..."
3,00004,001,Banashankari,"1st Floor, Annakuteera, 3rd Stage, B..."
4,00005,002,Basavanagudi,"10, 3rd Floor, Lakshmi Associates, G..."


In [5]:
# location_df['lat'], location_df['lng'] = zip(*location_df['address'].apply(lambda x: get_lat_lng(gmaps, x)))

In [56]:
Banashankari_geocode_result = gmaps.geocode('Banashankari,Bangalore')
Basavanagudi_geocode_result = gmaps.geocode('Basavanagudi,Bangalore')
Bangalore_geocode_result = gmaps.geocode('Bangalore')

In [55]:
Banashankari_geocode_result[0]['geometry']

{'bounds': {'northeast': {'lat': 12.9565439, 'lng': 77.57441370000001},
  'southwest': {'lat': 12.912888, 'lng': 77.531272}},
 'location': {'lat': 12.9254533, 'lng': 77.546757},
 'location_type': 'APPROXIMATE',
 'viewport': {'northeast': {'lat': 12.9565439, 'lng': 77.57441370000001},
  'southwest': {'lat': 12.912888, 'lng': 77.531272}}}

In [53]:
Bangalore_geocode_result[0]['geometry']

{'bounds': {'northeast': {'lat': 13.173706, 'lng': 77.8826809},
  'southwest': {'lat': 12.7342888, 'lng': 77.3791981}},
 'location': {'lat': 12.9715987, 'lng': 77.5945627},
 'location_type': 'APPROXIMATE',
 'viewport': {'northeast': {'lat': 13.173706, 'lng': 77.8826809},
  'southwest': {'lat': 12.7342888, 'lng': 77.3791981}}}

In [57]:
Basavanagudi_geocode_result[0]['geometry']

{'bounds': {'northeast': {'lat': 12.9572519, 'lng': 77.5801421},
  'southwest': {'lat': 12.9286136, 'lng': 77.5613754}},
 'location': {'lat': 12.9405997, 'lng': 77.5737633},
 'location_type': 'APPROXIMATE',
 'viewport': {'northeast': {'lat': 12.9572519, 'lng': 77.5801421},
  'southwest': {'lat': 12.9286136, 'lng': 77.5613754}}}

In [50]:
pd.unique(location_df['location_name'])

array(['Banashankari', 'Basavanagudi', 'Mysore Road', 'Jayanagar',
       'Kumaraswamy Layout', 'Rajarajeshwari Nagar', 'Vijay Nagar',
       'Uttarahalli', 'JP Nagar', 'South Bangalore', 'City Market',
       'Nagarbhavi', 'Bannerghatta Road', 'BTM', 'Kanakapura Road',
       'Bommanahalli', nan, 'CV Raman Nagar', 'Electronic City', 'HSR',
       'Marathahalli', 'Sarjapur Road', 'Wilson Garden', 'Shanti Nagar',
       'Koramangala 5th Block', 'Koramangala 8th Block', 'Richmond Road',
       'Koramangala 7th Block', 'Jalahalli', 'Koramangala 4th Block',
       'Bellandur', 'Whitefield', 'East Bangalore', 'Old Airport Road',
       'Indiranagar', 'Koramangala 1st Block', 'Frazer Town', 'RT Nagar',
       'MG Road', 'Brigade Road', 'Lavelle Road', 'Church Street',
       'Ulsoor', 'Residency Road', 'Shivajinagar', 'Infantry Road',
       'St. Marks Road', 'Cunningham Road', 'Race Course Road',
       'Commercial Street', 'Vasanth Nagar', 'HBR Layout', 'Domlur',
       'Ejipura', 'Jeevan 

In [ ]:
info_dict = {
    'Banashankari':,
    'Basavanagudi':,
    'Mysore Road':,
    'Jayanagar':,
    'Kumaraswamy Layout',
    'Rajarajeshwari Nagar',
    'Vijay Nagar',
    'Uttarahalli',
    'JP Nagar',
    'South Bangalore',
    'City Market',
    'Nagarbhavi',
    'Bannerghatta Road',
    'BTM',
    'Kanakapura Road',
    'Bommanahalli',
    'CV Raman Nagar',
    'Electronic City',
    'HSR',
    'Marathahalli',
    'Sarjapur Road',
    'Wilson Garden',
    'Shanti Nagar',
    'Koramangala 5th Block',
    'Koramangala 8th Block',
    'Richmond Road',
    'Koramangala 7th Block',
    'Jalahalli',
    'Koramangala 4th Block',
    'Bellandur',
    'Whitefield',
    'East Bangalore',
    'Old Airport Road',
    'Indiranagar',
    'Koramangala 1st Block',
    'Frazer Town',
    'RT Nagar',
    'MG Road',
    'Brigade Road',
    'Lavelle Road',
    'Church Street',
    'Ulsoor',
    'Residency Road',
    'Shivajinagar',
    'Infantry Road',
    'St. Marks Road',
    'Cunningham Road',
    'Race Course Road',
    'Commercial Street',
    'Vasanth Nagar',
    'HBR Layout',
    'Domlur',
    'Ejipura',
    'Jeevan Bhima Nagar',
    'Old Madras Road',
    'Malleshwaram',
    'Seshadripuram',
    'Kammanahalli',
    'Koramangala 6th Block',
    'Majestic',
    'Langford Town',
    'Central Bangalore',
    'Sanjay Nagar',
    'Brookefield',
    'ITPL Main Road, Whitefield',
    'Varthur Main Road, Whitefield',
    'KR Puram',
    'Koramangala 2nd Block',
    'Koramangala 3rd Block',
    'Koramangala',
    'Hosur Road',
    'Rajajinagar',
    'Banaswadi',
    'North Bangalore',
    'Nagawara',
    'Hennur',
    'Kalyan Nagar',
    'New BEL Road',
    'Jakkur',
    'Rammurthy Nagar',
    'Thippasandra',
    'Kaggadasapura',
    'Hebbal',
    'Kengeri',
    'Sankey Road',
    'Sadashiv Nagar',
    'Basaveshwara Nagar',
    'Yeshwantpur',
    'West Bangalore',
    'Magadi Road',
    'Yelahanka',
    'Sahakara Nagar',
    'Peenya'
}